In [1]:
%load_ext autoreload
%autoreload 2

The idea here is simple: run active learning experiments with different sampling strategies and save results for later analysis.

In [12]:
import math
import json

import torch
from tqdm.notebook import tqdm
from modAL.uncertainty import classifier_uncertainty, classifier_entropy, classifier_margin

from src import data, model, active_learning

# Preparing data and parameters

They would be the same for all experiments.

In [13]:
params = {
    'weight_regularization': 1e-5,
    'dropout_regularization': 1e-3,
}

x_train, y_train = data.prepare_data('./data/train_32x32.mat')
x_test, y_test = data.prepare_data('./data/test_32x32.mat')

initial_set_size = 1000
budget_step = 1000

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

active_learning_length = math.floor((len(x_train) * 0.8 - initial_set_size) / budget_step)

# Prediction uncertainty

Classifier uncertainty is 1 - P(highest prediction probability).

In [14]:
ac1 = active_learning.ActiveLearning(model.ConcreteDropoutCNN, params, x_train, y_train, x_test, y_test,
                                    classifier_uncertainty, initial_train_size=initial_set_size,
                                    sample_batch_size=budget_step, device=device)
for _ in tqdm(range(active_learning_length)):
    ac1.step()

with open('./results/classifier_uncertainty_test_results.json', 'w') as file:
    json.dump(ac1.active_learning_process, file, indent=2)

  0%|          | 0/36 [00:00<?, ?it/s]

# Prediction margin

Margin uncertainty is the difference between the probabilities of first and second most likely predictions.

In [ ]:
ac2 = active_learning.ActiveLearning(model.ConcreteDropoutCNN, params, x_train, y_train, x_test, y_test,
                                    classifier_margin, initial_train_size=initial_set_size,
                                    sample_batch_size=budget_step, device=device)
for _ in tqdm(range(active_learning_length)):
    ac2.step()

with open('./results/classifier_margin_test_results.json', 'w') as file:
    json.dump(ac2.active_learning_process, file, indent=2)

# Prediction entropy

Entropy of the class probabilities.

In [ ]:
ac3 = active_learning.ActiveLearning(model.ConcreteDropoutCNN, params, x_train, y_train, x_test, y_test,
                                    classifier_entropy, initial_train_size=initial_set_size,
                                    sample_batch_size=budget_step, device=device)
for _ in tqdm(range(active_learning_length)):
    ac3.step()

with open('./results/classifier_entropy_test_results.json', 'w') as file:
    json.dump(ac3.active_learning_process, file, indent=2)

  0%|          | 0/36 [00:00<?, ?it/s]

# Prediction entropy with Montecarlo dropout

Entropy of the average probabilities after multiple forward passes with dropout layers turned on.

In [ ]:
ac4 = active_learning.ActiveLearning(model.ConcreteDropoutCNN, params, x_train, y_train, x_test, y_test,
                                    active_learning.montecarlo_entropy, initial_train_size=initial_set_size,
                                    sample_batch_size=budget_step, device=device)
for _ in tqdm(range(active_learning_length)):
    ac4.step()

with open('./results/montecarlo_entropy_test_results.json', 'w') as file:
    json.dump(ac4.active_learning_process, file, indent=2)

In [ ]:
ac5 = active_learning.ActiveLearning(model.ConcreteDropoutCNN, params, x_train, y_train, x_test, y_test,
                                    active_learning.random_baseline, initial_train_size=initial_set_size,
                                    sample_batch_size=budget_step, device=device)
for _ in tqdm(range(active_learning_length)):
    ac5.step()

with open('./results/random_baseline_test_results.json', 'w') as file:
    json.dump(ac5.active_learning_process, file, indent=2)